# Test on Velodyne data

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from scipy.signal import savgol_filter
from scipy.spatial.transform import Rotation as R

from planeslam.scan import Scan, velo_pc_to_scan
from planeslam.general import pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.point_cloud import velo_preprocess

%load_ext autoreload
%autoreload 2

Flight room

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
PCs = []
for i in range(len(os.listdir(pcpath))):  
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
poses = []
for i in range(len(os.listdir(posepath))):  
    filename = posepath+'\pose_'+str(i)+'.npy'
    pose = np.load(filename)
    poses.append(pose)

In [ ]:
# Compute rover velocities (m/s) thru finite differencing
vels = 10 * np.diff(np.asarray(poses)[:,:3], axis=0)

# Apply some simple smoothing
vels[:,0] = savgol_filter(vels[:,0], 20, 3)
vels[:,1] = savgol_filter(vels[:,1], 20, 3)

# Transform velocities from world frame to local (should essentially result in all x)
local_vels = (quat_to_R(poses[i][3:]).T @ vels.T).T

vel_mags = np.linalg.norm(vels, axis=1)

In [ ]:
thetas = np.zeros(len(poses))
for i in range(len(poses)):
    r = R.from_quat(poses[i][3:])
    thetas[i] = r.as_euler('zyx')[0]

d_thetas = 10 * np.diff(thetas)  # angular velocities (rad/s)
d_thetas = d_thetas[np.abs(d_thetas) < 1]

In [ ]:
fig = go.Figure(go.Scatter(x=np.arange(len(d_thetas)), y=d_thetas))
fig.show()

In [ ]:
fig = go.Figure(go.Scatter(x=np.arange(len(vels)), y=vel_mags))
fig.show()

In [ ]:
frame_num = 283

In [ ]:
vel_mags[frame_num-1]

In [ ]:
d_thetas[frame_num-1]

In [ ]:
PC = PCs[frame_num]
data = go.Scatter3d(x=PC[:,0], y=PC[:,1], z=PC[:,2],
        mode='markers', marker=dict(size=2, color=np.arange(len(PC))), hovertext=np.arange(len(PC)))
#data1 = pc_plot_trace(PCs[156], color=np.arange(len(PCs[156])))
#data2 = pc_plot_trace(PCs[frame_num], color=np.arange(len(PCs[frame_num])))
fig = go.Figure(data=data)
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()